### Jupyter Notebook for MDPI itex glove data analysis
# Steps: 
1. Check if Python libraries listed in imports (I used Python 3.8)
2. Local copy of I-Tex glove dataset (Set variable dset_path in code cell below with dataset folder path in your PC)
3. Run cells sequentially to extract features from LG, RG 
4. Evaluate feature significance with SHAP 

## Todos:
1. Extract features suggested with MDPI reviewers [V,S] (Decrease in amplitude within activity, speed)
2. Train using cross validation? (Or do proper train-test file split) 

In [2]:
import pandas as pd
import glob
import random
import plotly.graph_objects as go
import numpy as np
import os
from scipy import signal
from scipy.fft import rfft, rfftfreq
from matplotlib import pyplot as plt
import re
import datetime
import scipy
import plotly.express as px
import math
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from itertools import cycle
import plotly.express as px
from matplotlib import pyplot as plt
import seaborn as sns; sns.set()
import time

#Hyper params
# Base path of dataset 
dset_path = "/Users/shehjarsadhu/Desktop/UniversityOfRhodeIsland/Graduate/WBL/Project_IOTEX/iotex-glove/PD"

np.random.seed(123)
prom_val= (0.01, 4)
width_val = (2,20)
Dist_val = 10

In [ ]:
import tsfel


In [3]:
def extract_peak_frequency(data, sampling_rate,n=1024):
    """Compute the peak frequency.

    Requires Scipy
    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    sampling_rate : float
      Sampling frequency of the data.
    n : float
      FFT points 
      (1024,2048)

    Return
    ------
    abs(peak_freq * sampling_rate) : float
      Peak frequency.
    """
    data = signal.detrend(data)
    fft_data = np.fft.fft(data,n)
#     freqs = np.fft.fftfreq(len(data))
    freqs = np.fft.fftfreq(n)

    peak_coefficient = np.argmax(np.abs(fft_data))
    peak_freq = freqs[peak_coefficient]
    return abs(peak_freq * sampling_rate)
    
def NormalizeData(data):
    '''
    Normalize data with max and min
    '''

    return (data - np.min(data)) / (np.max(data) - np.min(data))

def sampen(L, m, r):
    '''
    Sample entropy for given array L
    '''
    N = len(L)
    B = 0.0
    A = 0.0
    
    
    # Split time series and save all templates of length m
    xmi = np.array([L[i : i + m] for i in range(N - m)])
    xmj = np.array([L[i : i + m] for i in range(N - m + 1)])

    # Save all matches minus the self-match, compute B
    B = np.sum([np.sum(np.abs(xmii - xmj).max(axis=1) <= r) - 1 for xmii in xmi])

    # Similar for computing A
    m += 1
    xm = np.array([L[i : i + m] for i in range(N - m + 1)])

    A = np.sum([np.sum(np.abs(xmi - xm).max(axis=1) <= r) - 1 for xmi in xm])

    # Return SampEn
    return -np.log(A / B)


def peak_valley_analysis(data, Prominence=prom_val, Width = width_val,Distance=Dist_val):  
    '''
    Peak valley analysis for given data array, prominence value, width, minimum distance
    Uses signal.find_peaks    
    '''
    data = signal.detrend(data)
    peaks, properties = signal.find_peaks(data, prominence=Prominence, width=Width,distance=Distance)
    valley, properties = signal.find_peaks(data*-1, prominence=Prominence, width=Width,distance=Distance)        
            
    p2p_dist = np.diff(peaks)
    mean_p2p_dist = np.mean(p2p_dist)
    std_p2p_dist = np.std(p2p_dist)
    num_pk = len(peaks)
    num_val = len(valley)

#     if(num_pk == num_val):
#         p2v_dist = np.abs(peaks-valley)
#         mean_p2v_dist = np.mean(p2v_dist)
#         std_p2v_dist = np.std(p2v_dist)
#         p2v_height = np.abs(data[peaks]-data[valley])
#         mean_p2v_height = np.mean(p2v_height)
#         std_p2v_height = np.std(p2v_height)
#     elif(num_pk > num_val):
#         peaks = peaks[peaks<np.max(valley)]
#         peaks = peaks[:num_val]
#         p2v_dist = np.abs(peaks-valley)
#         mean_p2v_dist = np.mean(p2v_dist)
#         std_p2v_dist = np.std(p2v_dist)
#         p2v_height = np.abs(data[valley]-data[peaks])
#         mean_p2v_height = np.mean(p2v_height)
#         std_p2v_height = np.std(p2v_height)
#     elif(num_pk<num_val):
#         valley = valley[valley<np.max(peaks)]
#         valley = valley[:num_pk]
#         p2v_dist = np.abs(peaks-valley)
#         mean_p2v_dist = np.mean(p2v_dist)
#         std_p2v_dist = np.std(p2v_dist)
#         p2v_height = np.abs(data[peaks]-data[valley])
#         mean_p2v_height = np.mean(p2v_height)
#         std_p2v_height = np.std(p2v_height)
#     num_pk = len(peaks)
    return [mean_p2p_dist,std_p2p_dist]

def bandpower(data, band, fs=64,  method='welch', window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Requires MNE-Python >= 0.14.
    Parameters
    ----------
    data : 1d-array
      Input signal in the time-domain.
    fs : float
      Sampling frequency of the data.
    band : list
      Lower and upper frequencies of the band of interest.
    method : string
      Periodogram method: 'welch' or 'multitaper'
    window_sec : float
      Length of each window in seconds. Useful only if method == 'welch'.
      If None, window_sec = (1 / min(band)) * 2.
    relative : boolean
      If True, return the relative power (= divided by the total power of the signal).
      If False (default), return the absolute power.

    Return
    ------
    bp : float
      Absoluteor relative band power.
    """
    
#     from scipy.signal import welch
#     from scipy.integrate import simps
#     from mne.time_frequency import psd_array_multitaper

    band = np.asarray(band)
    low, high = band

    # Compute the modified periodogram (Welch)
    if method == 'welch':
        if window_sec is not None:
            nperseg = window_sec * fs
        else:
            nperseg = (2 / low) * fs

        freqs, psd = signal.welch(data, fs, nperseg=nperseg)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find index of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using parabola (Simpson's rule)
    bp = scipy.integrate.simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= scipy.integrate.simps(psd, dx=freq_res)
    return bp




def butter_bandpass(lowcut, highcut, fs, order=5):
        '''
        Butterworth filter for given low and high frequency, sampling rate (fs) and order
        '''
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = signal.butter(order, [low, high], analog=False, btype='band', output='sos')
        return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):

        sos = butter_bandpass(lowcut, highcut, fs, order=order)
        y = signal.sosfilt(sos, data)
        return y
    
#     if any(res.strftime('%Y%m%d-%H%M%S') in word for word in text_files):
#         print(res.strftime('%Y%m%d-%H%M%S') +' is there inside the list!')
#         return str(word)
#     else:
#         print(res.strftime('%Y%m%d-%H%M%S') +' is not there inside the list')
#         return 0
#     filter_object = filter(lambda a: res.strftime('%Y%m%d-%H%M%S') in a, text_files)
#     text_file  = [k for k in text_files if res.strftime('%Y%m%d-%H%M%S')+"_patient_form.txt" in k]
        

def stride_trick(a, stride_length, stride_step):
    """
    apply framing using the stride trick from numpy.

    Args:
        a (array) : signal array.
        stride_length (int) : length of the stride.
        stride_step (int) : stride step.

    Returns:
        blocked/framed array.
    """
    nrows = ((a.size - stride_length) // stride_step) + 1
    n = a.strides[0]
    return np.lib.stride_tricks.as_strided(a,
                                           shape=(nrows, stride_length),
                                           strides=(stride_step*n, n))


def framing(sig, fs=64, win_len=0.25, win_hop=0.125):
    """
    transform a signal into a series of overlapping frames (=Frame blocking).

    Args:
        sig     (array) : a normalized and detrended signal.
        fs        (int) : the sampling frequency of the signal we are working with.
                          Default is 64.
        win_len (float) : window length in sec.
                          Default is 0.025.
        win_hop (float) : step between successive windows in sec.
                          Default is 0.01.

    Returns:
        array of frames.
        frame length.

    Notes:
    ------
        Uses the stride trick to accelerate the processing.
    """
    # run checks and assertions
    if win_len < win_hop: print("ParameterError: win_len must be larger than win_hop.")

    # compute frame length and frame step (convert from seconds to samples)
    frame_length = win_len * fs
    frame_step = win_hop * fs
    signal_length = len(sig)
    frames_overlap = frame_length - frame_step

    # compute number of frames and left sample in order to pad if needed to make
    # sure all frames have equal number of samples  without truncating any samples
    # from the original signal
    rest_samples = np.abs(signal_length - frames_overlap) % np.abs(frame_length - frames_overlap)
    pad_signal = np.append(sig, np.array([0] * int(frame_step - rest_samples) * int(rest_samples != 0.)))

    # apply stride trick
    frames = stride_trick(pad_signal, int(frame_length), int(frame_step))
    return frames, frame_length


def _calculate_normalized_short_time_energy(frames):
    '''Calculates the short time FFT band energy'''

    return np.sum(np.abs(np.fft.rfft(a=frames, n=len(frames)))**2, axis=-1) / len(frames)**2


def naive_frame_energy_vad(sig, fs=64, threshold=0, win_len=0.25, win_hop=0.25, E0=1e7):
    '''
    For detecting signal activity (windowing activity data)
    '''
    # framing
    frames, frames_len = framing(sig=sig, fs=fs, win_len=win_len, win_hop=win_hop)

    # compute short time energies to get voiced frames
    energy = _calculate_normalized_short_time_energy(frames)
    log_energy = 10 * np.log10(energy / E0)

    # normalize energy to 0 dB then filter and format
    energy = scipy.signal.medfilt(log_energy, 5)
    energy = np.repeat(energy, frames_len)
#     print(energy.shape)
    mean_energy = np.mean(energy)
    threshold = np.average([mean_energy,np.amin(energy)])
    # compute vad and get speech frames
    vad     = np.array(energy > threshold, dtype=sig.dtype)
    #find and remove re-detection
    diff_vad = np.diff(vad)
#     print(threshold,np.where(diff_vad==-1)[0].shape)
    if(len(np.where(diff_vad==-1)[0])!=0):
#         print("here")
        pos_neg = np.where(diff_vad==-1)[0]
#         print(pos_neg)
        vad[int(pos_neg[0]):] = 0
#     print(threshold)


    vframes = np.array(frames.flatten()[np.where(vad==1)], dtype=sig.dtype)
    

    return energy, vad, np.array(vframes, dtype=np.float64)


def multi_plots(data, titles, fs, plot_rows, step=1, colors=["b", "r", "m", "g", "b", "y"]):
    # first fig
    plt.figure()
    plt.subplots(plot_rows, 1, figsize=(20, 10))
    plt.subplots_adjust(left=0.125, right=0.9, bottom=0.1, top=0.99, wspace=0.4, hspace=0.99)

    for i in range(plot_rows):
        plt.subplot(plot_rows, 1, i+1)
        y = data[i]
        plt.plot([i/fs for i in range(0, len(y), step)], y, colors[i])
        plt.gca().set_title(titles[i])
    plt.show()

    # second fig
    sig, vad = data[0], data[-2]
    # plot VAD and orginal signal
    plt.subplots(1, 1, figsize=(20, 10))
    plt.plot([i/fs for i in range(len(sig))], sig, label="Signal")
    plt.plot([i/fs for i in range(len(vad))], max(sig)*vad, label="VAD")
    plt.legend(loc='best')
    plt.show()

def zcr(sig):
    ''' 
    Calculates zero crossing rate of a given signal after detrending and normalizing
    '''
    sig = signal.detrend(sig)
    sig_norm = [sig- min(sig)]
    mval = max(abs(i) for i in sig)
    sig = np.array([i / mval for i in sig])
    val = ((sig[:-1] * sig[1:]) < 0).sum()
    return val

def mean_freq(sig, fs):
    '''
    Mean frequency within given signal
    '''    
    Y = np.abs(np.fft.rfft(sig)) 
    power_spectrum = Y**2
    freq = np.fft.rfftfreq(len(sig), fs)
    # Then calculate (weighted) mean
    return np.sum(freq * power_spectrum / np.sum(power_spectrum)) # This should equal to the formula in your image.

def rmsValue(sig):
    '''
    Finds root mean square of a given signal
    '''

    square = 0
    mean = 0.0
    root = 0.0
    n = len(sig)
     
    #Calculate square
    for i in range(0,n):
        square += (sig[i]**2)
     
    #Calculate Mean
    mean = (square / (float)(len(sig)))
     
    #Calculate Root
    root = math.sqrt(mean)
     
    return root
 
# Provide flex sigs for analysis, 
# returns [ft_mean_p2p_dist,ft_std_p2p_dist,ft_mean_p2v_dist,ft_std_p2v_dist,ft_mean_p2v_height,ft_std_p2v_height,ft_pdenergy,ft_dysenergy,ft_mean_freq, ft_rms
def kinetic_analysis(sig,fs,order,lf,hf,prom_val,width_val,Dist_val,detect_theshold=-20):
    '''
    Computes peak-valley locations and band energy within given flexion/inertial signal  (For hand flip, finger tap, finger to nose, hand open-close)

    TODO: Find features for start middle and ending windows

    '''
    sig = signal.detrend(sig)
    b, a = scipy.signal.butter(order, [lf*2/fs, hf*2/fs], 'band') 
    sig = scipy.signal.filtfilt(b, a, sig)
    sig_feats = []
    pdenergy = bandpower(sig, [5, 10], fs, 'welch',relative=True)
    dysenergy = bandpower(sig, [2,5], fs, 'welch',relative=True)
    energy, vad, detected = naive_frame_energy_vad(sig, fs=fs, threshold=detect_theshold,
                                                     win_len=0.25, win_hop=0.25)            
    sig_feats = peak_valley_analysis(detected, Prominence=prom_val, Width = width_val,Distance=Dist_val)
    sig_feats.append(pdenergy)
    sig_feats.append(dysenergy)
    sig_psd = bandpower(sig, [2, 20], fs, 'welch',relative=True) 
    sig_feats.append(sig_psd)
    sig_feats.append(mean_freq(sig, fs))
    sig_feats.append(rmsValue(detected))
                
    return sig_feats
    
                
    
                
# Provide inertial signals (acc[x,y,z] & gyr[x,y,z]) for either RH or HH task
# Returns seg1_acc_pdenergy,seg1_acc_dysenergy,seg1_acc_df,seg1_acc_mf,seg1_acc_std,seg1_acc_sent,seg1_acc_psd,seg1_acc_rms\,seg1_gyr_pdenergy,seg1_gyr_dys_energy,seg1_gyr_df,seg1_gyr_mf,seg1_gyr_std,seg1_gyr_sent,seg1_gyr_psd,seg1_gyr_rmsseg2_acc_pdenergy,seg2_acc_dysenergy,seg2_acc_df,seg2_acc_mf,seg2_acc_std,seg2_acc_sent,seg2_acc_psd,seg2_acc_rms\,seg2_gyr_pdenergy,seg2_gyr_dysenergy,seg2_gyr_df,seg2_gyr_mf,seg2_gyr_std,seg2_gyr_sent,seg2_gyr_psd,seg2_gyr_rms
def stationary_analysis(inertial_sigs,fs):
    '''
    Computes band energy and other signal features inertial signals  (For resting hands, hold out hands tasks)

    TODO: Find features for start middle and ending windows

    '''
    inertial_sigs = np.transpose(inertial_sigs)
#     print(inertial_sigs.shape)
    sig_seg_len = len(inertial_sigs[0])//2
    [x ** 2 for x in range(10) if x % 2 == 0]
    acc_rms_raw = np.array([rmsValue(inertial_sigs[:3,i]) for i in range(inertial_sigs.shape[1])])

#        np.sqrt(sum(inertial_sigs[0]**2))+ np.sqrt(sum(inertial_sigs[1]**2))+np.sqrt(sum(inertial_sigs[2]**2))
    gyr_rms_raw = np.array([rmsValue(inertial_sigs[3:,i]) for i in range(inertial_sigs.shape[1])])

    b, a = scipy.signal.butter(2, [1*2/fs, 30*2/fs], 'band') 
    # Based on : An automated methodology for levodopa-induced dyskinesia: Assessment based on gyroscope and accelerometer signals
    acc_rms = scipy.signal.filtfilt(b, a, acc_rms_raw)
    gyr_rms = scipy.signal.filtfilt(b, a, gyr_rms_raw)
    r_val = r_val = np.mean(np.std(acc_rms))
    sig_feats = []    
    seg1_acc_pdenergy = bandpower(acc_rms[:sig_seg_len], [5, 10], fs, 'welch',relative=True)
    sig_feats.append(seg1_acc_pdenergy)
    seg1_acc_dysenergy = bandpower(acc_rms[:sig_seg_len], [2, 5], fs, 'welch',relative=True)
    sig_feats.append(seg1_acc_dysenergy)
    seg1_acc_df = extract_peak_frequency(acc_rms[:sig_seg_len],64)
    sig_feats.append(seg1_acc_df)
    seg1_acc_mf = mean_freq(acc_rms[:sig_seg_len], fs)
    sig_feats.append(seg1_acc_mf)
    seg1_acc_std = np.std(acc_rms[:sig_seg_len])
    sig_feats.append(seg1_acc_std)
    seg1_acc_sent = sampen(acc_rms[:sig_seg_len], m=2, r=r_val)
    sig_feats.append(seg1_acc_sent)
    seg1_acc_psd = bandpower(acc_rms_raw[:sig_seg_len], [2, 20], fs, 'welch',relative=True) 
    sig_feats.append(seg1_acc_psd)
    
    seg1_gyr_pdenergy = bandpower(gyr_rms[:sig_seg_len], [2, 5], fs, 'welch',relative=True)
    sig_feats.append(seg1_gyr_pdenergy)
    seg1_gyr_dys_energy = bandpower(gyr_rms[:sig_seg_len], [5, 10], fs, 'welch',relative=True)
    sig_feats.append(seg1_gyr_dys_energy)
    seg1_gyr_df = extract_peak_frequency(gyr_rms[:sig_seg_len],fs)
    sig_feats.append(seg1_gyr_df)
    seg1_gyr_mf = mean_freq(gyr_rms[:sig_seg_len], fs)
    sig_feats.append(seg1_gyr_mf)
    seg1_gyr_std = np.std(gyr_rms[:sig_seg_len])
    sig_feats.append(seg1_gyr_std)
    seg1_gyr_sent = sampen(gyr_rms[:sig_seg_len], m=2, r=r_val)
    sig_feats.append(seg1_gyr_sent)
    seg1_gyr_psd = bandpower(gyr_rms_raw[:sig_seg_len], [2, 20], fs, 'welch',relative=True) 
    sig_feats.append(seg1_gyr_psd)

    seg2_acc_pdenergy = bandpower(acc_rms[sig_seg_len:], [2, 5], fs, 'welch',relative=True)
    sig_feats.append(seg2_acc_pdenergy)
    seg2_acc_dysenergy = bandpower(acc_rms[sig_seg_len:], [5, 10], fs, 'welch',relative=True)
    sig_feats.append(seg2_acc_dysenergy)
    seg2_acc_df = extract_peak_frequency(acc_rms[sig_seg_len:],fs)
    sig_feats.append(seg2_acc_df)
    seg2_acc_mf = mean_freq(acc_rms[sig_seg_len:], fs)
    sig_feats.append(seg2_acc_mf)
    seg2_acc_std = np.std(acc_rms[sig_seg_len:])
    sig_feats.append(seg2_acc_std)
    seg2_acc_sent = sampen(acc_rms[sig_seg_len:], m=2, r=r_val)
    sig_feats.append(seg2_acc_sent)
    seg2_acc_psd = bandpower(acc_rms_raw[sig_seg_len:], [2, 20], 64, 'welch',relative=True) 
    sig_feats.append(seg2_acc_psd)
    sig_acc_zc = zcr(acc_rms)
    sig_feats.append(sig_acc_zc)

    seg2_gyr_pdenergy = bandpower(gyr_rms[sig_seg_len:], [2, 5], 64, 'welch',relative=True)
    sig_feats.append(seg2_gyr_pdenergy)
    seg2_gyr_dysenergy = bandpower(gyr_rms[sig_seg_len:], [5, 10], 64, 'welch',relative=True)
    sig_feats.append(seg2_gyr_dysenergy)
    seg2_gyr_df = extract_peak_frequency(gyr_rms[sig_seg_len:],64)
    sig_feats.append(seg2_gyr_df)
    seg2_gyr_mf = mean_freq(gyr_rms[sig_seg_len:], 64)
    sig_feats.append(seg2_gyr_mf)
    seg2_gyr_std = np.std(gyr_rms[sig_seg_len:])
    sig_feats.append(seg2_gyr_std)
    seg2_gyr_sent = sampen(gyr_rms[sig_seg_len:], m=2, r=r_val)
    sig_feats.append(seg2_gyr_sent)
    seg2_gyr_psd = bandpower(gyr_rms_raw[sig_seg_len:], [2, 20], 64, 'welch',relative=True) 
    sig_feats.append(seg2_gyr_psd)
    sig_gyr_zc = zcr(gyr_rms)

    sig_feats.append(sig_gyr_zc)

    

    return sig_feats

                
    


                
    


In [7]:
cwd = os.getcwd()
# Print the current working directory
# print("Current working directory: {0}".format(cwd))
# Medication intake labels
med_usage = {
  "0_to_1_hrs": 0,
  "1_to_2_hrs": 1,
  "2_to_3_hrs": 2,
  "4_plus":3,
  "1_to_2hrs": 1,
}
channels = ['index']
cnames = ["ft_mean_p2p_dist","ft_std_p2p_dist",
#"ft_std_p2v_dist","ft_mean_p2v_height","ft_std_p2v_height",
"ft_pdenergy","ft_dysenergy","ft_psd","ft_mean_freq","ft_rms",
"hh_seg1_acc_pdenergy","hh_seg1_acc_dysenergy","hh_seg1_acc_df",
"hh_seg1_acc_mf","hh_seg1_acc_std","hh_seg1_acc_sent","hh_seg1_acc_psd",
"hh_seg1_gyr_pdenergy","hh_seg1_gyr_dysenergy",
"hh_seg1_gyr_df","hh_seg1_gyr_mf","hh_seg1_gyr_std","hh_seg1_gyr_sent",
"hh_seg1_gyr_psd","hh_seg2_acc_pdenergy",
"hh_seg2_acc_dysenergy","hh_seg2_acc_df","hh_seg2_acc_mf",
"hh_seg2_acc_std","hh_seg2_acc_sent","hh_seg2_acc_psd", "hh_acc_zc",
"hh_seg2_gyr_pdenergy","hh_seg2_gyr_dysenergy","hh_seg2_gyr_df","hh_seg2_gyr_mf","hh_seg2_gyr_std",
"hh_seg2_gyr_sent","hh_seg2_gyr_psd","hh_gyr_zc",
"rh_seg1_acc_pdenergy","rh_seg1_acc_dysenergy","rh_seg1_acc_df",
"rh_seg1_acc_mf","rh_seg1_acc_std","rh_seg1_acc_sent","rh_seg1_acc_psd",
"rh_seg1_gyr_pdenergy","rh_seg1_gyr_dysenergy",
"rh_seg1_gyr_df","rh_seg1_gyr_mf","rh_seg1_gyr_std","rh_seg1_gyr_sent",
"rh_seg1_gyr_psd","rh_seg2_acc_pdenergy",
"rh_seg2_acc_dysenergy","rh_seg2_acc_df","rh_seg2_acc_mf",
"rh_seg2_acc_std","rh_seg2_acc_sent","rh_seg2_acc_psd","rh_acc_zc",
"rh_seg2_gyr_pdenergy","rh_seg2_gyr_dysenergy","rh_seg2_gyr_df","rh_seg2_gyr_mf","rh_seg2_gyr_std",
"rh_seg2_gyr_sent","rh_seg2_gyr_psd","rh_gyr_zc", 
"fn_mean_p2p_dist","fn_std_p2p_dist",#"fn_mean_p2v_dist",
# "fn_std_p2v_dist","fn_mean_p2v_height","fn_std_p2v_height",
"fn_pdenergy","fn_dysenergy","fn_psd","fn_mean_freq","fn_rms",
"hf_mean_p2p_dist","hf_std_p2p_dist",#"hf_mean_p2v_dist",
#"hf_std_p2v_dist","hf_mean_p2v_height","hf_std_p2v_height",
"hf_pdenergy","hf_dysenergy","hf_psd","hf_mean_freq","hf_rms","fno","pno","med_status"]
feat_rg = []
feat_rg_tsfresh = []
feat_rg_tsfresh_col = []
for i in range(4):
    pno = i+1
    print("Part No: {}".format(pno))
    # csv_files = glob.glob('C:/Users/dan95/Desktop/Participant1/**/*.csv',recursive=True)
    dset_csv_fpath = dset_path +'/Participant'+str(pno)+'/**/*.csv'
    label_text_fpath = dset_path +'/Participant'+str(pno)+'/**/*.txt'

    csv_files = glob.glob(dset_csv_fpath,recursive=True)
    text_files = glob.glob(label_text_fpath,recursive=True)
    rg_files = [s for s in csv_files if "rg_" in s]
    lg_files = [s for s in csv_files if "lg_" in s]
    lg_acts = [0,2,4,6,7,9]
    rg_acts = [1,3,5,6,8,9]
    print("RG cnt: {}, LG cnt: {}, Txt cnt: {}".format(len(rg_files),len(lg_files),len(text_files)))# print(sorted(csv_files))
    # print(glob.glob(csv_files))
    # plot_channels = ['index','thumb','pitch','roll']
    rg_act_no = 1
    dist_ft = 0
    tot_files = len(rg_files)
    print("Total RG files: {}".format(tot_files))
    lf = 1
    hf = 10
    b, a = scipy.signal.butter(2, [lf*2/64, hf*2/64], 'band')
    # plot results
    # multi_plots(data=[sig, energy, vad, detected],
    #             titles=["Input signal (activity + idle)", "Short time energy",
    #                     "Activity detection", "Detected signal"],
    #             fs=64, plot_rows=4, step=1)


    # Extract RG FT, HH, RH task features (Mean Peak Distance, Std of distance, Mean Peak Height, Std of height, PSD band energy[2-10Hz])
    # Activity codes:
    # 0: Left hand finger tap
    # 1: Right hand finger tap
    # 2: Left hand open close
    # 3: Right hand open close
    # 4: Left hand flip
    # 5: Right hand flip 	
    # 6: Both hands out
    # 7: Left finger to nose
    # 8: Right finger to nose	
    # 9: Hold out hand

#     df_rg = pd.DataFrame([],columns =cnames)
    for file_num in range(0,tot_files):
        ch_rg = rg_files[file_num] #random.choice(rg_files)
        df_rg = pd.read_csv(ch_rg,)
        ch_form = text_files[file_num]     #Open questionnaire and find medication intake period 
        my_file = open(ch_form, "r")
        medication_status = med_usage[my_file.read().split("\n")[4].split("=")[1]]
        print("File No: {}, Med: {}" .format(file_num,medication_status))

    #     sig = scipy.signal.filtfilt(b, a, df_rg[df_rg['activity'] == rg_act_no][channels].values)
#         try:
        index_sig = df_rg[df_rg['activity'] == rg_act_no]['index'].values
        if len(index_sig)< 256:
            print("File too small, skipping")
            continue
        ## FT section
        sig_feats = kinetic_analysis(index_sig,64,order=2,lf=1,hf=30, prom_val= (0.01, None), width_val = (2,20) ,Dist_val=5,detect_theshold=0)
#         tsfresh_feats = extract_features(df_rg[["index","pitch","roll"]],
#                      default_fc_parameters=settings,
#                      # we impute = remove all NaN features automatically
#                      impute_function=impute)
#         feat_rg_tsfresh = tsfresh_feats_ft.columns
        print("FT feat len - {} ".format(len(sig_feats)))
        ch_ = ["ax","ay","az","gx","gy","gz"]
        # sig = butter_bandpass_filter(df_rg[df_rg['activity'] == rg_act_no]['index'].values,0.1,30,64)
        ## Extact features for HH task in sections 1 & 2 for accel and gyro [rms accel, dominant freq, mean freq,  ]
        hh_feats = stationary_analysis(df_rg[df_rg['activity'] == 6][ch_].values,64)
#             print("HH feat len - {} ".format(len(hh_feats)))
        sig_feats.extend(hh_feats)
#             print("tot feat len - {} ".format(len(sig_feats)))

         ## Extact features for RH task in sections 1 & 2 for accel and gyro [rms accel, dominant freq, mean freq,  ]
        rh_feats = stationary_analysis(df_rg[df_rg['activity'] == 9][ch_].values,64)
#             print("RH feat len - {} ".format(len(rh_feats)))
        sig_feats.extend(rh_feats)
        fn_feats = kinetic_analysis(df_rg[df_rg['activity'] == 8]['pitch'],64,order=2,lf=1,hf=20, prom_val= (0.05,None), width_val = (None,20) ,Dist_val=5,detect_theshold=0)
        sig_feats.extend(fn_feats)
        hf_feats = kinetic_analysis(df_rg[df_rg['activity'] == 5]['pitch'],64,order=2,lf=0.1,hf=20, prom_val= (0.05,None), width_val = (None, 20) ,Dist_val=5,detect_theshold=0)
        sig_feats.extend(hf_feats)

#             print("tot feat len - {} ".format(len(sig_feats)))
        sig_feats.append(file_num)
#             print("tot feat len - {} ".format(len(sig_feats)))
        sig_feats.append(pno)
#             print("tot feat len - {} ".format(len(sig_feats)))
        sig_feats.append(medication_status)
        print("tot feat len - {} ".format(len(sig_feats)))
        feat_rg.append(sig_feats)
#         feat_rg_ts_fresh.append(tsfresh_feats_ft.values)
        print("Total RG feat len - {} ".format(np.array(feat_rg).shape))

            #         print(peak_valley_feats)
#         except ValueError:
#             print("Error in: {}".format(file_num))
#             continue
feat_arr = np.transpose(feat_rg)
print(len(feat_rg))
# df_rg = pd.DataFrame(feat_rg, columns = cnames)

Part No: 1
RG cnt: 21, LG cnt: 21, Txt cnt: 20
Total RG files: 21
File No: 0, Med: 3
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (1, 84) 
File No: 1, Med: 3
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (2, 84) 
File No: 2, Med: 3
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (3, 84) 
File No: 3, Med: 3
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (4, 84) 
File No: 4, Med: 2
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (5, 84) 
File No: 5, Med: 3
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (6, 84) 
File No: 6, Med: 3
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (7, 84) 
File No: 7, Med: 2
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (8, 84) 
File No: 8, Med: 3
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (9, 84) 
File No: 9, Med: 3
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (10, 84) 
File No: 10, Med: 1
FT feat len - 7 
tot feat len - 84 
Total RG feat len - (11, 84) 
File No

IndexError: list index out of range

In [ ]:
cwd = os.getcwd()
# Print the current working directory
# print("Current working directory: {0}".format(cwd))
med_usage = {
  "0_to_1_hrs": 0,
  "1_to_2_hrs": 1,
  "2_to_3_hrs": 2,
  "4_plus":3,
  "1_to_2hrs": 1,
}
channels = ['index']

feat_lg = []

for i in range(4):
    pno = i+1
    print("Part No: {}".format(pno))
    dset_csv_fpath = dset_path +'/Participant'+str(pno)+'/**/*.csv'
    label_text_fpath = dset_path +'/Participant'+str(pno)+'/**/*.txt'

    csv_files = glob.glob(dset_csv_fpath,recursive=True)
    text_files = glob.glob(label_text_fpath,recursive=True)
    rg_files = [s for s in csv_files if "rg_" in s]
    lg_files = [s for s in csv_files if "lg_" in s]
    lg_acts = [0,2,4,6,7,9]
    rg_acts = [1,3,5,6,8,9]
    print("RG cnt: {}, LG cnt: {}, Txt cnt: {}".format(len(rg_files),len(lg_files),len(text_files)))# print(sorted(csv_files))
    # print(glob.glob(csv_files))
    # plot_channels = ['index','thumb','pitch','roll']
    rg_act_no = 1
    dist_ft = 0
    tot_files = len(rg_files)
    lf = 1
    hf = 10
    b, a = scipy.signal.butter(2, [lf*2/64, hf*2/64], 'band')
    
    # plot results
    # multi_plots(data=[sig, energy, vad, detected],
    #             titles=["Input signal (activity + idle)", "Short time energy",
    #                     "Activity detection", "Detected signal"],
    #             fs=64, plot_rows=4, step=1)


    # Extract RG FT, HH, RH task features (Mean Peak Distance, Std of distance, Mean Peak Height, Std of height, PSD band energy[2-10Hz])

#     df_rg = pd.DataFrame([],columns =cnames)
    for file_num in range(0,tot_files):
        ch_lg = lg_files[file_num] #random.choice(rg_files)
        df_lg = pd.read_csv(ch_lg)
        ch_form = text_files[file_num]     #Open questionnaire and find medication intake period 
        my_file = open(ch_form, "r")
        medication_status = med_usage[my_file.read().split("\n")[4].split("=")[1]]
        print("File No: {}, Med: {}" .format(file_num,medication_status))

    #     sig = scipy.signal.filtfilt(b, a, df_rg[df_rg['activity'] == rg_act_no][channels].values)
        try:
            index_sig = df_lg[df_lg['activity'] == 0]['index'].values
            if len(index_sig)< 256:
                print("File too small, skipping")
                continue
            ## FT section
            sig_feats = kinetic_analysis(index_sig,64,order=2,lf=1,hf=30, prom_val= (0.01, None), width_val = (2,20) ,Dist_val=5,detect_theshold=0)
            
            print("FT feat len - {} ".format(len(sig_feats)))
            ch_ = ["ax","ay","az","gx","gy","gz"]
            # sig = butter_bandpass_filter(df_rg[df_rg['activity'] == rg_act_no]['index'].values,0.1,30,64)
            ## Extact features for HH task in sections 1 & 2 for accel and gyro [rms accel, dominant freq, mean freq,  ]
            hh_feats = stationary_analysis(df_lg[df_lg['activity'] == 6][ch_].values,64)
    #             print("HH feat len - {} ".format(len(hh_feats)))
            sig_feats.extend(hh_feats)
    #             print("tot feat len - {} ".format(len(sig_feats)))

             ## Extact features for RH task in sections 1 & 2 for accel and gyro [rms accel, dominant freq, mean freq,  ]
            rh_feats = stationary_analysis(df_lg[df_lg['activity'] == 9][ch_].values,64)
#             print("RH feat len - {} ".format(len(rh_feats)))
            sig_feats.extend(rh_feats)
            fn_feats = kinetic_analysis(df_lg[df_lg['activity'] == 7]['pitch'],64,order=2,lf=1,hf=20, prom_val= (0.05,None), width_val = (None,20) ,Dist_val=5,detect_theshold=0)
            sig_feats.extend(fn_feats)
            hf_feats = kinetic_analysis(df_lg[df_lg['activity'] == 4]['pitch'],64,order=2,lf=1,hf=20, prom_val= (0.05,None), width_val = (None,20) ,Dist_val=5,detect_theshold=0)
            sig_feats.extend(hf_feats)

#             print("tot feat len - {} ".format(len(sig_feats)))
            sig_feats.append(file_num)
#             print("tot feat len - {} ".format(len(sig_feats)))
            sig_feats.append(pno)
#             print("tot feat len - {} ".format(len(sig_feats)))
            sig_feats.append(medication_status)
            print("tot feat len - {} ".format(len(sig_feats)))
            feat_lg.append(sig_feats)
            print("Total LG feat len - {} ".format(np.array(feat_lg).shape))

            #         print(peak_valley_feats)
        except ValueError:
            print("Error in: {}".format(file_num))
            continue
feat_arr = np.transpose(feat_lg)
print(len(feat_lg))
# df_rg = pd.DataFrame(feat_rg, columns = cnames)

In [10]:
df_rg = pd.DataFrame(feat_rg, columns = cnames)
# df_lg = pd.DataFrame(feat_lg, columns = cnames)

In [11]:
df_rg

,ft_mean_p2p_dist,ft_std_p2p_dist,ft_pdenergy,ft_dysenergy,ft_psd,ft_mean_freq,ft_rms,hh_seg1_acc_pdenergy,hh_seg1_acc_dysenergy,hh_seg1_acc_df,...,hf_mean_p2p_dist,hf_std_p2p_dist,hf_pdenergy,hf_dysenergy,hf_psd,hf_mean_freq,hf_rms,fno,pno,med_status
0,18.542857,1.872246,0.169749,0.637038,0.802474,0.001006,0.030138,0.177060,0.302064,1.9375,...,15.196078,9.048929,0.175781,0.565126,0.698766,0.000583,30.742855,0,1,3
1,21.264706,2.810786,0.158823,0.729557,0.920990,0.000916,0.065461,0.151240,0.273827,2.3750,...,12.714286,4.802210,0.190682,0.624434,0.805955,0.000631,47.697743,1,1,3
2,21.307692,2.445864,0.160514,0.645094,0.861767,0.000954,0.044316,0.181298,0.231411,1.4375,...,13.800000,6.774462,0.203529,0.626800,0.817210,0.000715,36.810795,2,1,3
3,17.861111,2.043319,0.213856,0.786169,0.954184,0.001133,0.050456,0.132858,0.166840,2.0000,...,10.947368,4.968042,0.177191,0.618259,0.751977,0.000705,38.456086,3,1,3
4,18.033333,3.646764,0.186048,0.682106,0.841126,0.001099,0.040875,0.233973,0.443467,3.1875,...,12.507246,6.800731,0.193604,0.561389,0.738626,0.000699,31.348363,4,1,2
5,17.800000,2.625516,0.193527,0.795809,0.942052,0.001024,0.046174,0.170787,0.281354,1.3750,...,9.545455,3.055952,0.171977,0.513473,0.636579,0.000565,43.211824,5,1,3
6,19.851852,2.414519,0.187977,0.705833,0.917697,0.001034,0.055281,0.194333,0.292310,1.0625,...,11.238636,4.400399,0.205217,0.635454,0.821048,0.000805,38.849348,6,1,3
7,18.535714,2.542245,0.175146,0.750267,0.916869,0.001044,0.041782,0.182083,0.205445,3.6875,...,13.000000,7.080882,0.189068,0.517631,0.694674,0.000620,24.268899,7,1,2
8,19.615385,1.665878,0.172959,0.774509,0.945940,0.000996,0.042491,0.185899,0.401506,1.6875,...,11.533333,4.769580,0.236900,0.571546,0.888568,0.000978,36.490258,8,1,3
9,19.054054,1.469435,0.187165,0.508450,0.781223,0.001129,0.025375,0.145910,0.288215,1.6250,...,12.738095,6.572162,0.191547,0.584175,0.768518,0.000504,30.732896,9,1,3


In [ ]:

# df_rg = df_rg(df_rg['med_status']==0)
# df_rg = df_rg.loc[df_rg['med_status'] != 0]
# df_lg = df_lg.loc[df_lg['med_status'] != 0]
# print(df_rg['med_status'].value_counts())
# df_rg = df_rg.loc[df_rg['med_status'] != 2]
# df_lg = df_lg.loc[df_lg['med_status'] != 2]
# 
# df_rg = df_rg[df_rg['med_status']>0]
# df_lg = df_lg[df_lg['med_status']>0]

# Set medication status to binary variable 0-3hrs = 1 [on meds], 4+hrs = 0 [off meds]

df_rg.loc[(df_rg['med_status'] <3,'med_status')]=1
df_rg.loc[(df_rg['med_status'] ==3,'med_status')]=0
df_lg.loc[(df_lg['med_status'] <3,'med_status')]=1
df_lg.loc[(df_lg['med_status'] ==3,'med_status')]=0
print(df_rg['med_status'].value_counts())
print(df_lg['med_status'].value_counts())



# df_rg.loc[(df_rg['med_status'] <3,'med_status')]=1
# df_rg.loc[(df_rg['med_status'] ==3,'med_status')]=0
# df_lg.loc[(df_lg['med_status'] <3,'med_status')]=1
# df_lg.loc[(df_lg['med_status'] ==3,'med_status')]=0


# Compute relative features

df_rg["rel_seg1_acc_energy"] = df_rg["rh_seg1_acc_psd"]/df_rg["hh_seg1_acc_psd"]
df_rg["rel_seg1_gyr_energy"] = df_rg["rh_seg1_gyr_psd"]/df_rg["hh_seg1_gyr_psd"]
df_rg["rel_seg2_acc_energy"] = df_rg["rh_seg2_acc_psd"]/df_rg["hh_seg2_acc_psd"]
df_rg["rel_seg2_gyr_energy"] = df_rg["rh_seg2_gyr_psd"]/df_rg["hh_seg2_gyr_psd"]
df_rg["rel_acc_zc"] = df_rg["rh_acc_zc"]/df_rg["hh_acc_zc"]
df_rg["rel_gyr_zc"] = df_rg["rh_gyr_zc"]/df_rg["hh_gyr_zc"]
df_rg["rel_ft_energy"] = df_rg["ft_dysenergy"]/df_rg["ft_pdenergy"]

df_lg["rel_seg1_acc_energy"] = df_lg["rh_seg1_acc_psd"]/df_lg["hh_seg1_acc_psd"]
df_lg["rel_seg1_gyr_energy"] = df_lg["rh_seg1_gyr_psd"]/df_lg["hh_seg1_gyr_psd"]
df_lg["rel_seg2_acc_energy"] = df_lg["rh_seg2_acc_psd"]/df_lg["hh_seg2_acc_psd"]
df_lg["rel_seg2_gyr_energy"] = df_lg["rh_seg2_gyr_psd"]/df_lg["hh_seg2_gyr_psd"]
df_lg["rel_acc_zc"] = df_lg["rh_acc_zc"]/df_lg["hh_acc_zc"]
df_lg["rel_gyr_zc"] = df_lg["rh_gyr_zc"]/df_lg["hh_gyr_zc"]
df_lg["rel_ft_energy"] = df_lg["ft_dysenergy"]/df_lg["ft_pdenergy"]


In [ ]:
df_lg.columns

In [5]:
# Save to file if needed
# df_rg.to_csv(dset_path+'/rg_feats.csv')
# df_lg.to_csv((dset_path+'/lg_feats.csv')


In [6]:
## SHAP Analysis (right glove) for computing feature importance

feat_set_rg = df_rg
feat_set_rg = feat_set_rg.drop(['fno', 'pno'], axis=1) 
feat_set_rg = feat_set_rg.fillna(0)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer


# Spliiting data into test and train sets



X_train, X_test, y_train, y_test = train_test_split(feat_set_rg.drop('med_status', axis=1), feat_set_rg['med_status'], test_size=0.20, random_state=4)
# fitting the model
scaler = Normalizer().fit(feat_set_rg.drop('med_status', axis=1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

model = RandomForestClassifier(n_estimators=700, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)
# plotting feature importances
features = feat_set_rg.drop('med_status', axis=1).columns
importances = model.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(10,15))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()
print( [features[i] for i in indices])

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test,plot_type="bar",
               show=False)
fig = plt.gcf() # gcf means "get current figure"
plt.rcParams['font.size'] = '14'
plt.rcParams["font.weight"] = "bold"
plt.rcParams['axes.labelsize'] = '14'
plt.rcParams['legend.fontsize'] = '14'


ax = plt.gca() #gca means "get current axes"
# leg = ax.legend(bbox_to_anchor=(0., 1.02, 1., .102))
# for l in leg.get_texts(): l.set_text(l.get_text().replace('Class', 'Klasse'))
plt.show()

# perm_importance = permutation_importance(model, X_test, y_test)
# sorted_idx = perm_importance.importances_mean.argsort()
# plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
# plt.xlabel("Permutation Importance")


KeyError: "['fno' 'pno'] not found in axis"

In [ ]:
## SHAP Analysis (left glove) for computing feature importance

feat_set_rg = df_lg
feat_set_rg = feat_set_rg.drop(['fno', 'pno'], axis=1) 
feat_set_rg = feat_set_rg.fillna(0)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import shap
from sklearn.inspection import permutation_importance


# Spliiting data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(feat_set_rg.drop('med_status', axis=1), feat_set_rg['med_status'], test_size=0.20, random_state=4)
# fitting the model
model = RandomForestClassifier(n_estimators=700, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)
# plotting feature importances
features = feat_set_rg.drop('med_status', axis=1).columns
importances = model.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(10,15))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()
print( [features[i] for i in indices])

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test,plot_type="bar",
               show=False)
fig = plt.gcf() # gcf means "get current figure"
plt.rcParams['font.size'] = '14'
plt.rcParams["font.weight"] = "bold"
plt.rcParams['axes.labelsize'] = '14'
plt.rcParams['legend.fontsize'] = '14'


ax = plt.gca() #gca means "get current axes"
# leg = ax.legend(bbox_to_anchor=(0., 1.02, 1., .102))
# for l in leg.get_texts(): l.set_text(l.get_text().replace('Class', 'Klasse'))
plt.show()

# perm_importance = permutation_importance(model, X_test, y_test)
# sorted_idx = perm_importance.importances_mean.argsort()
# plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
# plt.xlabel("Permutation Importance")


In [ ]:
cls.fit(X_train, y_train)
predictions = cls.predict(X_test)


In [ ]:
X_test

In [ ]:
feat_set_lg = df_lg
feat_set_lg = feat_set_lg.drop(['fno', 'pno'], axis=1) 
feat_set_lg = feat_set_lg.fillna(0)

# Spliiting data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(feat_set_lg.drop('med_status', axis=1), feat_set_lg['med_status'], test_size=0.20, random_state=4)
# fitting the model
model = RandomForestClassifier(n_estimators=700, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)
# plotting feature importances
features = feat_set_lg.drop('med_status', axis=1).columns
importances = model.feature_importances_
indices = np.argsort(importances)
plt.figure(figsize=(10,15))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

print( [features[i] for i in indices])


explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test,plot_type="bar",
               show=False)
fig = plt.gcf() # gcf means "get current figure"
plt.rcParams['font.size'] = '14'
plt.rcParams["font.weight"] = "bold"
plt.rcParams['axes.labelsize'] = '14'
plt.rcParams['legend.fontsize'] = '14'


ax = plt.gca() #gca means "get current axes"
# leg = ax.legend(bbox_to_anchor=(0., 1.02, 1., .102))
# for l in leg.get_texts(): l.set_text(l.get_text().replace('Class', 'Klasse'))
plt.show()


In [ ]:
corr = feat_set_rg.drop('med_status', axis=1).corr() # Generate correlation matrix

fig = go.Figure()
fig.add_trace(
    go.Heatmap(
        x = corr.columns,
        y = corr.index,
        z = np.array(corr)
    )
)

# corr2 = feat_set_lg.drop('med_status', axis=1).corr() # Generate correlation matrix

# fig2 = go.Figure()
# fig2.add_trace(
#     go.Heatmap(
#         x = corr2.columns,
#         y = corr2.index,
#         z = np.array(corr2)
#     )
# )


In [ ]:
corr2 = feat_set_lg.drop('med_status', axis=1).corr() # Generate correlation matrix
fig2 = go.Figure()
fig2.add_trace(
    go.Heatmap(
        x = corr2.columns,
        y = corr2.index,
        z = np.array(corr2)
    )
)


In [ ]:
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False

selected_columns_rg = feat_set_rg.drop('med_status', axis=1).columns[columns]
df_rg_select = feat_set_rg[selected_columns_rg]



In [ ]:
# P-value code
selected_columns_rg = selected_columns_rg[1:]
import statsmodels.api as sm

def backwardElimination(x, Y, sl, columns):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    
    regressor_OLS.summary()
    return x, columns
SL = 0.05
data_modeled, selected_columns_rg = backwardElimination(df_rg_select.iloc[:,1:].values, df_rg_select.iloc[:,0].values, SL, selected_columns_rg)
# data_modeled, selected_columns = backwardElimination(feat_set_rg.drop('med_status', axis=1).values, feat_set_rg['med_status'], SL, selected_columns)
result = pd.DataFrame()
result['med_status'] = feat_set_rg['med_status']
print(len(selected_columns_rg))
df_rg_select = pd.DataFrame(data = data_modeled, columns = selected_columns_rg)


In [ ]:
fig = plt.figure(figsize = (25, 25))
j = 0
for i in selected_columns_rg:
    plt.subplot(10, 8, j+1)
    j += 1
    sns.distplot(feat_set_rg[i][result['med_status']==0], color='g', label = 'Pre Medication')
    sns.distplot(feat_set_rg[i][result['med_status']==1], color='r', label = 'Post Medication')
    plt.legend(loc='best')
fig.suptitle('RG Data Analysis')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()


In [ ]:
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns_lg = feat_set_lg.drop('med_status', axis=1).columns[columns]
df_lg_select = feat_set_lg[selected_columns_lg]
selected_columns_lg = selected_columns_lg[1:]
# P-value code
SL = 0.05
data_modeled, selected_columns_lg = backwardElimination(df_lg_select.iloc[:,1:].values, df_lg_select.iloc[:,0].values, SL, selected_columns_lg)
# data_modeled, selected_columns = backwardElimination(feat_set_rg.drop('med_status', axis=1).values, feat_set_rg['med_status'], SL, selected_columns)
result = pd.DataFrame()
result['med_status'] = feat_set_lg['med_status']
df_lg_select = pd.DataFrame(data = data_modeled, columns = selected_columns_lg)


In [ ]:
fig = plt.figure(figsize = (25, 25))
j = 0
for i in data_lg.columns:
    plt.subplot(10, 8, j+1)
    j += 1
    sns.distplot(data_lg[i][result['med_status']==0], color='g', label = 'Pre Medication')
    sns.distplot(data_lg[i][result['med_status']==1], color='r', label = 'Post Medication')
    plt.legend(loc='best')
fig.suptitle('LG Data Analysis')
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.show()


In [ ]:
feat_set_rg_new

In [ ]:

df_rg[df_rg['activity'] == 6][ch_]
fig = px.box(data_frame = df_lg
       ,x = 'pno'
       ,y = 'hf_mean_p2p_dist'
       ,color= 'med_status'
       ,points="all"
       )


fig.update_layout(
#     legend=dict(
#     yanchor="bottom",
#     y=0.0,
#     xanchor="right",
# #     x=0.01
#     ),
    title={
        'text': "Left Glove - Hand Flip Mean Peak to Peak distance",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'bottom'},
    xaxis_title="Participant",
    yaxis_title="Sample difference",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="Black"
    ))

fig.show()


In [ ]:
fig = px.violin(df_lg, y="hf_mean_p2p_dist", x="pno", color="med_status")
fig.show()

In [ ]:
# Load dataset
# tips = sns.load_dataset('tips')

# Define some hatches
hatches = cycle(['///', 'x'])

# Boxplot
ax = sns.boxplot(x="pno", y="hf_mean_p2p_dist", hue="med_status", data=df_lg)
for i, patch in enumerate(ax.artists):
    # Boxes from left to right
    hatch = next(hatches)
    patch.set_hatch(hatch)

ax.axes.set_title("Left Hand Flip peak to peak distance", fontsize=16)
ax.set_xlabel("Participants", fontsize=14)
ax.set_ylabel("Mean peak to peak distance", fontsize=14)
ax.legend([],[], frameon=False)

plt.show()

In [ ]:
fig = px.violin(df_lg, y="rh_seg2_gyr_mf", x="pno", color="med_status")
fig.show()

In [ ]:
# Load dataset
# tips = sns.load_dataset('tips')

# Define some hatches
hatches = cycle(['///', 'x'])

# Boxplot
ax = sns.boxplot(x="pno", y="rh_seg2_gyr_mf", hue="med_status", data=df_rg)
for i, patch in enumerate(ax.artists):
    # Boxes from left to right
    hatch = next(hatches)
    patch.set_hatch(hatch)

ax.axes.set_title("Right Hand resting hand mean frequency", fontsize=16)
ax.set_xlabel("Participants", fontsize=14)
ax.set_ylabel("Mean Frequency (Hz)", fontsize=14)
# ax.legend([],[], frameon=False)
plt.show()

In [ ]:


fig.update_layout(
#     legend=dict(
#     yanchor="bottom",
#     y=0.0,
#     xanchor="right",
# #     x=0.01
#     ),
    title={
        'text': "Left Glove - Hand Flip Mean Peak to Peak distance",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'bottom'},
    xaxis_title="Participant",
    yaxis_title="Relative Energy (a.u)",
    font=dict(
        family="Times New Roman, monospace",
        size=18,
        color="Black"
    ))

fig.show()


In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rel_acc_zc'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rh_seg2_acc_mf'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_lg
       ,x = 'pno'
       ,y = 'hf_std_p2p_dist'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rh_seg2_gyr_psd'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'hh_seg2_gyr_psd'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rh_seg2_gyr_df'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rh_seg2_gyr_df'
       ,color= 'med_status'
#        ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'hh_seg2_gyr_df'
       ,color= 'med_status'
#        ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'hh_seg2_acc_df'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rh_seg2_gyr_df'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_lg
       ,x = 'pno'
       ,y = 'hh_seg2_gyr_df'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_lg
       ,x = 'pno'
       ,y = 'hf_pdenergy'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_lg
       ,x = 'pno'
       ,y = 'hf_pdenergy'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rel_seg2_gyr_energy'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_lg
       ,x = 'pno'
       ,y = 'rel_seg2_gyr_energy'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rel_seg2_gyr_energy'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rh_seg2_acc_psd'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_lg
       ,x = 'pno'
       ,y = 'rh_seg2_acc_psd'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rh_seg2_gyr_psd'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'rel_seg1_gyr_energy'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'hh_seg2_acc_dysenergy'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'hh_gyr_zc'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
px.box(data_frame = df_rg
       ,x = 'pno'
       ,y = 'hh_seg1_mean_freq'
       ,color= 'med_status'
       ,points="all"
       )

In [ ]:
feat_rg[0]

In [ ]:
df_rg

In [ ]:
df_feat_rg = pd.DataFrame(feat_rg,columns = ['ax','ay','az','gx','gy','gz','index','thumb','middle'])


In [ ]:
plt.plot(acc_rms)